In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import timeit
#!pip install torchattacks
import torchattacks
import torch
from torchvision import io

In [2]:
#!pip install git+https://github.com/RobustBench/robustbench.git
import robustbench
from robustbench.utils import load_model,clean_accuracy
from robustbench.data import load_cifar10

# Load models and data

In [3]:
import os
#os.chdir('D:\\PFE black\\3.Robustbench and torchattacks')
os.getcwd()

'D:\\PFE black\\Transfer-attacks-comparison'

In [4]:
from cifar10_models.resnet import resnet50
my_model = resnet50(pretrained=True)
my_model.eval() 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
images_png = [name for name in os.listdir("D:\\PFE black\\4. Skip gradient method cifar10\\skip-connections-matter\\adv_images_cifar10") if os.path.splitext(name)[-1] == '.png']
images_png = [int(v.split(".")[0][1:]) for v in images_png]
images_png.sort()
images_png = ["n"+str(v)+".png.png"for v in images_png]

In [6]:
n_images = len(images_png)
adv_images_SGM = torch.zeros(n_images, 3, 32, 32, dtype=torch.uint8)

for i, image_png in enumerate(images_png):
    adv_images_SGM[i] = io.read_image(os.path.join("D:\\PFE black\\4. Skip gradient method cifar10\\skip-connections-matter\\adv_images_cifar10", image_png))
    
#make sure to change type of tensor to float and normalize between 0 and 1
adv_images_SGM = adv_images_SGM.float() / 255

In [7]:
# from torchvision import transforms
# preprocess = transforms.Compose([
#     transforms.Normalize(mean = [0.4914, 0.4822, 0.4465], std = [0.2471, 0.2435, 0.2616])
# ])
# adv_images_SGM = preprocess(adv_images_SGM)
# plt.imshow(adv_images_SGM[0,:,:,:].permute(1, 2, 0) )

In [8]:
dict_y_test = np.load("cifar10_image_to_label.npy", allow_pickle = True)[()]
y_test = torch.Tensor(list(dict_y_test.values()))
y_test = y_test.type(torch.LongTensor)
y_test

tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7,
        8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 3,
        8, 7, 6, 2, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 6, 6, 8, 4, 8, 4,
        5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 4, 0, 5, 8,
        0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0,
        7, 9, 8, 2, 7, 6, 9, 4, 3, 9, 6, 4, 7, 6, 5, 1, 5, 8, 8, 0, 4, 0, 5, 5,
        1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 7,
        0, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 5, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0,
        7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0,

In [9]:
model_rank1 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf')

In [10]:
model_rank2 = load_model(model_name='Gowal2021Improving_70_16_ddpm_100m', dataset='cifar10', threat_model='Linf')

In [11]:
model_rank3 = load_model(model_name='Gowal2020Uncovering_70_16_extra', dataset='cifar10', threat_model='Linf')

In [12]:
model_rank4 = load_model(model_name='Rebuffi2021Fixing_106_16_cutmix_ddpm', dataset='cifar10', threat_model='Linf')

In [13]:
model_rank5 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_ddpm', dataset='cifar10', threat_model='Linf')

In [14]:
model_vanilla = load_model(model_name='Standard', dataset='cifar10', threat_model='Linf')

# torchattacks

### SGM

In [15]:
SGM_results = []

In [16]:
acc = clean_accuracy(model_rank1, adv_images_SGM, y_test)
print('Robust Acc rank1: %2.2f %%'%(acc*100))
SGM_results.append(acc)

Robust Acc rank1: 90.00 %


In [17]:
acc = clean_accuracy(model_rank2, adv_images_SGM, y_test)
print('Robust Acc rank2: %2.2f %%'%(acc*100))
SGM_results.append(acc)

Robust Acc rank2: 87.00 %


In [18]:
acc = clean_accuracy(model_rank3, adv_images_SGM, y_test)
print('Robust Acc rank3: %2.2f %%'%(acc*100))
SGM_results.append(acc)

Robust Acc rank3: 88.60 %


In [19]:
acc = clean_accuracy(model_rank4, adv_images_SGM, y_test)
print('Robust Acc rank4: %2.2f %%'%(acc*100))
SGM_results.append(acc)

Robust Acc rank4: 86.20 %


In [20]:
acc = clean_accuracy(model_rank5, adv_images_SGM, y_test)
print('Robust Acc rank5: %2.2f %%'%(acc*100))
SGM_results.append(acc)

Robust Acc rank5: 85.80 %


In [21]:
acc = clean_accuracy(model_vanilla, adv_images_SGM, y_test)
print('Robust Acc vanilla: %2.2f %%'%(acc*100))
SGM_results.append(acc)

Robust Acc vanilla: 6.60 %


In [22]:
SGM_results

[0.9, 0.87, 0.886, 0.862, 0.858, 0.066]